In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import logging
logging.basicConfig(level=logging.INFO)
np.random.seed(0)

In [3]:
from replay_identification.simulate import make_simulated_data

(is_replay, speed, linear_distance,
 power, spikes, time, multiunit) = make_simulated_data()

INFO:spectral_connectivity.transforms:Multitaper(sampling_frequency=1500, time_halfbandwidth_product=1,
           time_window_duration=0.02, time_window_step=0.0006666666666666666,
           detrend_type='constant', start_time=0, n_tapers=1)


In [4]:
from replay_identification import ReplayDetector

detector = ReplayDetector(replay_speed=1, movement_var=4.0, place_bin_size=2.0, spike_model_knot_spacing=15,
                   spike_model_penalty=0.5, movement_state_transition_type='random_walk',
                   discrete_state_transition_type='constant')
detector.fit(is_replay, speed, linear_distance, power, spikes, multiunit)

INFO:replay_identification.decoders:Fitting speed model...
INFO:replay_identification.decoders:Fitting LFP power model...
INFO:replay_identification.decoders:Fitting spiking model...
INFO:replay_identification.decoders:Fitting multiunit model...


electrodes:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:replay_identification.decoders:Fitting continuous state transition...
INFO:replay_identification.decoders:Fitting discrete state transition...


ReplayDetector(discrete_diagonal=None, discrete_state_transition_penalty=1e-05,
               discrete_state_transition_type='constant',
               infer_track_interior=True, is_track_interior=None,
               lfp_model=<class 'sklearn.mixture._bayesian_mixture.BayesianGaussianMixture'>,
               lfp_model_kwargs={'max_iter': 200, 'n_components': 1,
                                 'tol': 1e-06},
               movement_state_transition_type='random_wa...
               multiunit_model_kwargs={'bandwidth': array([24., 24., 24., 24.,  6.,  6.])},
               multiunit_occupancy_kwargs={'bandwidth': array([6., 6.])},
               multiunit_occupancy_model=<class 'replay_identification.multiunit_likelihood.NumbaKDE'>,
               place_bin_size=2.0, position_range=None, replay_speed=1,
               speed_knots=None, speed_threshold=4.0,
               spike_model_knot_spacing=15, spike_model_penalty=0.5)

In [5]:
acausal_results = detector.predict(
    speed, linear_distance, power, spikes, time=time, use_likelihoods=['spikes'],
    use_acausal=True)

INFO:replay_identification.decoders:Predicting spikes likelihood...


neurons:   0%|          | 0/8 [00:00<?, ?it/s]

neurons:   0%|          | 0/8 [00:00<?, ?it/s]

INFO:replay_identification.decoders:Finding causal non-local probability and position...
INFO:replay_identification.decoders:Finding acausal non-local probability and position...


In [10]:
from replay_identification.discrete_state_transition import estimate_discrete_state_transition

estimate_discrete_state_transition(detector, acausal_results)

array([[9.99969987e-01, 3.00133793e-05],
       [1.99912678e-02, 9.80008732e-01]])

In [8]:
detector.discrete_state_transition_(np.arange(1))[0]

array([3.0e-05, 9.8e-01])

In [9]:
detector.discrete_state_transition_

functools.partial(<function _constant_probability at 0x7ff7eb7b64c0>, diagonal=array([3.0e-05, 9.8e-01]))

In [15]:
fit_args = dict(
    is_ripple=is_replay,
    speed=speed,
    position=linear_distance,
    lfp_power=power,
    spikes=spikes,
)
predict_args = dict(
    speed=speed,
    position=linear_distance,
    lfp_power=power,
    spikes=spikes,
    time=time,
    use_likelihoods=['spikes'],
    use_acausal=True
)
results, data_log_likelihoods = detector.estimate_parameters(
    fit_args, predict_args, estimate_likelihood=False)

INFO:replay_identification.decoders:Fitting speed model...
INFO:replay_identification.decoders:Fitting LFP power model...
INFO:replay_identification.decoders:Fitting spiking model...
INFO:replay_identification.decoders:Fitting continuous state transition...
INFO:replay_identification.decoders:Fitting discrete state transition...
INFO:replay_identification.decoders:Predicting spikes likelihood...


neurons:   0%|          | 0/8 [00:00<?, ?it/s]

neurons:   0%|          | 0/8 [00:00<?, ?it/s]

INFO:replay_identification.decoders:Finding causal non-local probability and position...
INFO:replay_identification.decoders:Finding acausal non-local probability and position...
INFO:replay_identification.decoders:iteration 0, likelihood: -0.7073872813347013
INFO:replay_identification.decoders:Predicting spikes likelihood...


neurons:   0%|          | 0/8 [00:00<?, ?it/s]

neurons:   0%|          | 0/8 [00:00<?, ?it/s]

INFO:replay_identification.decoders:Finding causal non-local probability and position...
INFO:replay_identification.decoders:Finding acausal non-local probability and position...
INFO:replay_identification.decoders:iteration 1, likelihood: -0.7073903858947136, change: -3.104560012290314e-06
